In [1]:
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [17]:
class SphericalCoordinate:
    def __init__(self, mesh, origin=None):
        self.mesh = mesh
        self.mesh.compute_triangle_normals()
        self.mesh.compute_vertex_normals()
        self.pointcloud = self.mesh.sample_points_uniformly(number_of_points=20000)
        if origin is None:
            self.origin = self.pointcloud.get_center()
        else:
            self.origin = origin
        self.pointcloud.translate(-self.origin)
        self.points = np.asarray(self.pointcloud.points)
        self.normals = np.asarray(self.pointcloud.normals)
        self.kd_tree = o3d.geometry.KDTreeFlann(self.pointcloud)
        
    def compute_batch_cos_similarity(self,vec):
        points = self.points.copy()
        points[:,0] *= vec[0]
        points[:,1] *= vec[1]
        points[:,2] *= vec[2]
        dot_points = points.sum(axis=1)
        points_norm = np.linalg.norm(vec) * np.linalg.norm(self.points, axis=1)
        return dot_points/points_norm

    def get(self, theta, phi):
        # Need to get the closest point from center to the shell.
        dir_vec = np.array([np.sin(phi)*np.cos(theta), np.sin(phi) * np.sin(theta), np.cos(phi)])
        cos_sim = self.compute_batch_cos_similarity(dir_vec)
        index = cos_sim.argmax()
        return self.points[index], self.normals[index]

    def get_wrapped(self, theta, x1):
        phi = np.arccos(x1)
        return self.get(theta, phi)

    def cartesian2spherical(self, points):
        # get rid of offset
        points[:,0] -= self.origin[0]
        points[:,1] -= self.origin[1]
        points[:,2] -= self.origin[2]
        sp_coord = np.zeros((len(points), 2))
        sp_coord[:,0] = np.arctan2(points[:,1], points[:,0]) # theta
        sp_coord[:,1] = np.arccos(points[:,2]/np.linalg.norm(points,axis=1)) # psi
        return sp_coord

def project2sphere(sp_coord, color=[0.5, 0.5, 0.5], scale=0.05):
    cart_coord = np.zeros((len(sp_coord), 3))
    cart_coord[:,0] = np.sin(sp_coord[:,1]) * np.cos(sp_coord[:,0])
    cart_coord[:,1] = np.sin(sp_coord[:,1]) * np.sin(sp_coord[:,0])
    cart_coord[:,2] = np.cos(sp_coord[:,1])
    cart_coord *= scale
    pcd = o3d.geometry.PointCloud()
    color = np.array(color)
    pcd.points = o3d.utility.Vector3dVector(cart_coord)
    pcd.colors = o3d.utility.Vector3dVector(np.tile(color, len(sp_coord)).reshape(-1,3))
    return cart_coord, pcd

In [18]:
# create mesh block
mesh_box = o3d.geometry.TriangleMesh.create_box(0.4, 0.4, 0.1)
mesh_box.translate([-0.2, -0.2, -0.05])
point_box = mesh_box.sample_points_uniformly(number_of_points=20000)
point_box.colors = o3d.utility.Vector3dVector(np.ones((20000, 3)) * 0.6)

In [19]:
box = SphericalCoordinate(mesh_box)

In [20]:
# visualize on a unit sphere
finger_region = np.load("../data/pcd_region/pcd_fingertip.npz")
thumb_points = finger_region['thumb']
index_points = finger_region['index']
middle_points = finger_region['middle']
ring_points = finger_region['ring']

In [21]:
_, thumb_pcd = project2sphere(box.cartesian2spherical(thumb_points), color=[0,0,0])
_, index_pcd = project2sphere(box.cartesian2spherical(index_points), color=[1,0,0])
_, middle_pcd = project2sphere(box.cartesian2spherical(middle_points), color=[0,1,0])
_, ring_pcd = project2sphere(box.cartesian2spherical(ring_points), color=[0,0,1])

In [22]:
ref_sphere = o3d.geometry.TriangleMesh.create_sphere(0.05)
o3d.visualization.draw_geometries([point_box, ref_sphere, thumb_pcd, index_pcd, middle_pcd, ring_pcd])

In [8]:
index_points

array([[ 0.20040162,  0.07369438, -0.00342802],
       [ 0.20040162,  0.15738069, -0.01726806],
       [ 0.20040162,  0.10091833,  0.00041782],
       ...,
       [ 0.0666762 ,  0.12571196, -0.05003288],
       [ 0.05517007,  0.09154843, -0.05003288],
       [ 0.13945832,  0.15882543, -0.05003288]])

In [11]:
middle_points

array([[ 0.20080324,  0.07508622, -0.0034609 ],
       [ 0.20080324,  0.15877253, -0.01730094],
       [ 0.20080324,  0.10231016,  0.00038494],
       ...,
       [ 0.06707782,  0.1271038 , -0.05006575],
       [ 0.0555717 ,  0.09294027, -0.05006575],
       [ 0.13985994,  0.16021726, -0.05006575]])